In [1]:
import numpy as np
import pandas as pd

# 1)Loading the Dataset and Segmenting It


In [2]:
df = pd.read_csv('MLChallengeData.csv')

df

,Status,Id,Refreshing,Runner,Baker,Counter,Regulator,Has Python,Has Whiteboard,Has Reached Balmers Peak,DNE,Type of Activity Id,Type of Movie Id,Type of GPU Id,Type of Laptop Id,Type of Toaster Id,Type of Deck Id,Type of Whale Id,Type of Star Id,Type of Dog Id
0,NaN,25,0,0,0,0,0,1,1,0,0,0.3,0.4,0.2,0.4,0.3,0.3,0.5,0.2,0.4
1,1.0,38,0,0,0,0,0,1,1,1,0,0.3,0.4,0.2,0.4,0.3,0.2,0.1,0.2,0.1
2,1.0,53,0,0,0,0,0,1,1,1,0,0.3,0.3,0.3,0.5,0.3,0.4,0.5,0.2,0.4
3,0.0,61,0,0,0,0,0,0,0,0,0,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,0.0,70,0,0,0,0,0,1,1,1,0,0.3,0.6,0.2,0.4,0.3,0.2,0.5,0.2,0.4
5,1.0,79,0,0,0,0,0,1,1,0,1,0.3,0.4,0.4,0.3,0.2,0.2,0.1,0.3,0.1
6,0.0,100,0,0,0,0,0,1,1,0,0,0.3,0.6,0.2,0.6,0.3,0.3,0.5,0.2,0.4
7,NaN,140,0,0,0,0,0,0,0,0,0,0.3,0.6,0.2,0.2,0.2,0.2,0.1,0.3,0.1
8,1.0,164,0,0,0,0,0,1,1,1,0,0.3,0.5,0.3,0.3,0.3,0.2,0.1,0.2,0.1
9,0.0,199,0,0,0,0,0,1,1,1,1,0.3,0.3,0.4,0.3,0.2,0.4,0.5,0.2,0.4


There should be 3218 candidates as listed in the challenge instructions 1075 negative, 1075 positive

In [3]:
known = df[df.Status.notnull()]
known.shape

(3218, 20)

There should be 32426 candidates in undetermined status, this will be what we classify

In [4]:
unknown = df[df.Status.isnull()]
unknown.shape

(32426, 20)

In [5]:
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [6]:
y = known['Status']
#x is our training parameters and does not include ID as it should not have anything to do with the decision made
x = known.drop(['Status','Id'],axis=1) 

In [7]:
#set 5% of our data as test and the other as train
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.05)

# Random Forest

iterate and test out different hyperparameters

Utilizing random search and lots of iterations I am going to try to narrow down the hyperparamaters

In [8]:


n_estimators = [1,2,4,8,16,50,100,250,500,750,1000,1500,2000]
max_features = ['auto', 'sqrt','log2']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10, 20]
min_samples_leaf = [1, 2, 4, 8]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
random_grid

{'n_estimators': [1, 2, 4, 8, 16, 50, 100, 250, 500, 750, 1000, 1500, 2000],
 'max_features': ['auto', 'sqrt', 'log2'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'min_samples_split': [2, 5, 10, 20],
 'min_samples_leaf': [1, 2, 4, 8],
 'bootstrap': [True, False]}

In [9]:
rf = RF()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 150, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(x_train,y_train)

Fitting 5 folds for each of 150 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:  9.1min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=150, n_jobs=-1,
          param_distributions={'n_estimators': [1, 2, 4, 8, 16, 50, 100, 250, 500, 750, 1000, 1500, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10, 20], 'min_samples_leaf': [1, 2, 4, 8], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [10]:
rf_random.best_params_

{'n_estimators': 250,
 'min_samples_split': 2,
 'min_samples_leaf': 8,
 'max_features': 'auto',
 'max_depth': 60,
 'bootstrap': True}

now next step is to use grid search to get the best estimate i can get and narrow down to those best hyper parameters

In [11]:
#grid search for best features narrowed down from the list above
param_grid = {
    'bootstrap': [True,False],
    'max_depth': [30],
    'max_features': ['sqrt'],
    'min_samples_leaf': [8],
    'min_samples_split': [20],
    'n_estimators': [int(x) for x in np.linspace(start = 10, stop = 30, num = 10)]
}
param_grid

{'bootstrap': [True, False],
 'max_depth': [30],
 'max_features': ['sqrt'],
 'min_samples_leaf': [8],
 'min_samples_split': [20],
 'n_estimators': [10, 12, 14, 16, 18, 21, 23, 25, 27, 30]}

In [12]:
rf2 = RF()
grid_search = GridSearchCV(estimator = rf2, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   16.8s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'bootstrap': [True, False], 'max_depth': [30], 'max_features': ['sqrt'], 'min_samples_leaf': [8], 'min_samples_split': [20], 'n_estimators': [10, 12, 14, 16, 18, 21, 23, 25, 27, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

Below should hopefully be my best parameters

In [13]:
myparams=grid_search.best_params_

In [14]:
myparams={'bootstrap': True,
 'max_depth': 30,
 'max_features': 'sqrt',
 'min_samples_leaf': 8,
 'min_samples_split': 20,
 'n_estimators': 12}

In [15]:
best_rf = grid_search.best_estimator_
trainscore = best_rf.score(x_train,y_train)
testscore = best_rf.score(x_test,y_test)
testscore,trainscore

(0.6894409937888198, 0.689892051030422)

In [16]:
meteRF = RF(n_estimators = myparams.get('n_estimators'),bootstrap=myparams.get('bootstrap'),max_depth=myparams.get('max_depth'),max_features=myparams.get('max_features'),min_samples_leaf=myparams.get('min_samples_leaf'),min_samples_split=myparams.get('min_samples_split'))
meteRF.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=8, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=12, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Next I am going to get the hyperparameters for the SVC and see how my best solution performs there using the same steps as above

# SVC
iterate and test out different hyperparameters

Set Up parameters for random search

In [17]:
random_grid_SVC = {'C': [1,2,4,10,50],
               'kernel': ['rbf','sigmoid','linear','poly'],
               'shrinking':[True,False],
               'probability':[True,False]}

doing less iterations than rf since my computer was not able to handle 750 fits 

In [18]:
svc = SVC()
svc_random = RandomizedSearchCV(estimator = svc, param_distributions = random_grid_SVC, n_iter = 50, cv = 5, verbose=2, random_state=42, n_jobs = -1)
svc_random.fit(x_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  3.4min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params=None, iid=True, n_iter=50, n_jobs=-1,
          param_distributions={'C': [1, 2, 4, 10, 50], 'kernel': ['rbf', 'sigmoid', 'linear', 'poly'], 'shrinking': [True, False], 'probability': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

due to large processing time of SVC, I will not be performing gridsearch

In [19]:
myparams_svc=svc_random.best_params_
best_svc = svc_random.best_estimator_
trainscore_svc = best_svc.score(x_train,y_train)
testscore_svc = best_svc.score(x_test,y_test)
testscore_svc,trainscore_svc

(0.6956521739130435, 0.6643768400392541)

Takin a quick peek at parameters

In [20]:
myparams_svc

{'shrinking': True, 'probability': False, 'kernel': 'poly', 'C': 2}

In [21]:
meteSVC = SVC(C = myparams_svc.get('C'),kernel=myparams_svc.get('kernel'),shrinking=myparams_svc.get('shrinking'),probability=myparams_svc.get('probability'))
meteSVC.fit(x_train,y_train)

SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

# Overall Comparison and Analysis

Confusion Matrix for Both

In [22]:
svc_predict = meteSVC.predict(x_test)
svc_predict

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [23]:
rf_predict = meteRF.predict(x_test)
rf_predict

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

## RF Confusion Matrix and F1 Score

In [24]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, rf_predict)

array([[107,   5],
       [ 46,   3]], dtype=int64)

In [25]:
from sklearn.metrics import f1_score
f1_score(y_test,rf_predict)

0.10526315789473682

#h2 SVC Confusion Matrix and F1 Score

In [26]:
confusion_matrix(y_test, svc_predict)

array([[112,   0],
       [ 49,   0]], dtype=int64)

In [27]:
f1_score(y_test,svc_predict)

C:\Users\Mete\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0

# Prediction and Saving Predictions

In [33]:
ids = unknown['Id'].values
#x is our training parameters and does not include ID as it should not have anything to do with the decision made
x_unknown = unknown.drop(['Status','Id'],axis=1) 

rf_predict = meteRF.predict(x_unknown)

write1 = np.column_stack(ids)
write2 = np.column_stack(rf_predict)

write_final=np.row_stack((write1,write2))

out_dev = pd.DataFrame(data=write_final.T,columns=['ids','pred_status'])

out = pd.DataFrame(columns=['ids','pred_status'])
out[['ids','pred_status']] = out_dev[['ids','pred_status']].astype(int)

In [36]:
out.to_csv('metemorris_results.csv',index=False)

In [34]:
out

,ids,pred_status
0,25,0
1,140,0
2,319,0
3,329,0
4,361,0
5,396,0
6,594,0
7,611,0
8,613,0
9,628,0
